In [42]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import sklearn as skl
import nltk
import csv
import networkx as nx
import operator
import matplotlib.pyplot as plt
import pandas as pd
import re
from collections import defaultdict

In [43]:
node_inf_raw = pd.read_csv("./node_information.csv")
node_inf = node_inf_raw.values
all_auth = np.array([(re.sub(r',Jr.?', 'Jr',
                            re.sub(r',(?=[a-z])', '',
                                   re.sub(r'\([^)]*\)?', '',
                                          re.sub(r' ', '' , a[3])))).split(","),a[0]) if type(a[3]) == str else [] for a in node_inf])
print(all_auth[:100])


[(['PaulS.Aspinwall'], 1001L) (['M.Cvetic', 'H.Lu', 'C.N.Pope'], 1002L)
 (['Y.S.Myung', 'GungwonKang'], 1003L) (['AdamD.Helfer'], 1004L)
 (['J.Fuchs', 'C.Schweigert'], 1005L) (['RudolfHaag'], 1006L)
 (['ShengLi', 'YongZhang', 'ZhongyuanZhu'], 1007L)
 (['DonaldSpector'], 1008L) (['GourangaC.Nayak', 'WalterGreiner'], 1009L)
 (['A.V.Belitsky', 'S.Voren', 'P.vanNieuwenhuizen'], 1010L)
 (['TsunehideKuroki'], 1011L)
 (['V.Fateev', 'A.Zamolodchikov', 'Al.Zamolodchikov'], 1012L)
 (['RainerDick', 'DzoMikulovicz'], 1013L) (['Franz-MarcBoas'], 1014L)
 (['AliceRogers'], 1015L) (['EnriqueAlvarez', 'CesarGomez'], 1016L)
 (['R.S.Ward'], 1017L) list([])
 (['I.Klich', 'J.Feinberg', 'A.Mann', 'M.Revzen'], 1019L)
 (['STTsou'], 1020L)
 (['ZalanHorvath', 'RobertL.Karp', 'LaszloPalla'], 1021L)
 (['BogdanDamski'], 1022L) list([])
 (['JeromeP.Gauntlett', 'GaryW.Gibbons', 'ChristopherM.Hull', 'PaulK.'], 1024L)
 (['BrianR.Greene', 'C.I.Lazaroiu'], 1025L)
 (['AllenC.Hirshfeld', 'ThomasSchwarzweller'], 1026L)
 ([

In [60]:
authors = defaultdict(list)
for a,c in zip(all_auth,node_inf):
    if a != []:
        #print(a)
        (b,i) = a
        for auteur in b:
            authors[auteur].append(i)

del authors['']

print(authors)

defaultdict(<type 'list'>, {'H.Samtleben': [6034L, 10076L, 103032L, 106153L, 112035L, 207206L, 9710210L, 9804152L, 9903111L, 9912111L], 'DanijelJurman': [304006L], 'Rui-hongYue': [109091L], 'ChristianR\\"omelsberger': [2037L, 3263L], 'RaikoP.Zaikov': [9303087L, 9304075L, 9308016L, 9308017L], 'S.A.Antonenko': [9803264L], 'A.Sevrin': [10151L, 11018L, 11264L, 101018L, 105274L, 9209037L, 9303051L, 9303058L, 9306033L, 9306059L, 9306135L, 9403183L, 9408033L, 9412198L, 9706218L, 9801080L, 9812207L, 9905141L], 'TiloWettig': [9605110L, 9704055L, 9811044L], 'V.Bazhanov': [9412229L, 9812091L, 9812247L], 'A.Gerasimov': [5053L, 9405011L, 9601161L], 'A.Minzoni': [9606092L], 'AlexerYu.Kamenshchik': [6220L, 6221L, 10288L, 9601006L, 9603021L, 9604182L, 9604194L, 9606132L, 9609178L, 9610075L, 9707138L, 9707168L, 9802059L, 9803036L], 'AndreiBytsenko': [9303061L, 9405090L], 'P.Windey': [9704190L], 'SujayK.Ashok': [211291L], 'Sergio.A.Pernice': [9612125L], 'V.Frolov': [12048L, 12252L, 12260L, 212115L, 3022

In [61]:
nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kingr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
print(node_inf[1][1])

2000


In [63]:
def graph_articles(citation_set, node_info, directed_or_not = 'n'):
    if directed_or_not == 'y':
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    for node in node_info:
        G.add_node(node[0], year = node[1])
    for i in citation_set:
        if i[2] == '1':
            if G.node[i[0]]['year'] < G.node[i[1]]['year']:
                G.add_edge(i[0], i[1])
            else:
                G.add_edge(i[1], i[0])
    return G

In [64]:
G = graph_articles(training_set, node_info, directed_or_not = 'y')

In [83]:
L = [4,2,3,6,1]
L.sort()
print(L)

[1, 2, 3, 4, 6]


In [87]:
authors_h = dict()
for aut in authors.keys():
    liste = [G.in_degree(str(i)) for i in authors[aut]]
    liste.sort(reverse=True)
    i=1
    j=0
    print(liste)
    while j<len(liste) and liste[j] >= i:
        i+=1
        j+=1
        
    print(i-1)
    authors_h[aut] = i-1

print(authors_h)
        
#Le dictionnaire qui renvoie a chaque auteur son h-index
#Il ne reste plus qu'à associer à chaque node la moyenne de l'h-index de ses auteurs
#La feature sera ensuite calculée comme le page rank/club: addition des h_index des 2 nodes

[18, 14, 11, 11, 10, 8, 7, 5, 4, 4]
7
[11]
1
[25]
1
[25, 15]
2
[3, 3, 3, 1]
3
[2]
1
[20, 19, 18, 18, 14, 10, 10, 10, 9, 8, 7, 7, 7, 7, 2, 1, 1, 0]
9
[14, 10, 4]
3
[7, 6, 2]
2
[30, 8, 7]
3
[3]
1
[9, 7, 7, 5, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0]
4
[2, 0]
1
[16]
1
[66]
1
[2]
1
[21, 16, 11, 11, 9, 8, 8, 8, 4, 3, 3, 3, 3]
8
[3]
1
[]
0
[6, 5, 4, 1]
3
[4]
1
[41, 29, 19, 15, 14, 14, 14, 13, 13, 12, 11, 10, 8, 6, 6, 5, 5, 5, 4, 4, 3, 2, 1, 0, 0]
11
[10, 8, 7, 3, 2]
3
[12, 5, 2, 1, 1]
2
[1, 0]
1
[14, 14, 10, 7, 6]
5
[1, 0]
1
[48, 40, 18, 17]
4
[3]
1
[0]
0
[5, 0]
1
[6, 5, 4, 3, 3, 0, 0]
3
[8, 6]
2
[34, 13]
2
[2, 1]
1
[0]
0
[21, 14, 14, 11]
4
[9, 6, 4, 4, 3, 3, 2, 2, 2, 1, 0, 0]
4
[29, 26, 21, 19, 15, 11, 11, 11, 10, 8, 5, 5, 5, 1]
9
[12, 10]
2
[44, 42, 29, 25, 12, 10, 7, 7, 7, 3, 1, 1]
7
[0]
0
[0]
0
[35, 23, 16, 10, 8, 1]
5
[2]
1
[37, 27, 11, 5]
4
[3]
1
[1]
1
[52, 22, 16, 9]
4
[21, 4, 1]
2
[11]
1
[4]
1
[11, 7]
2
[29, 28, 24, 20, 13, 12, 11, 10, 4]
8
[7, 1, 1, 0, 0]
1
[8, 7, 6]
3
[5, 4, 4, 1]
3
[10]
1
[4

15
[28, 26, 14, 12, 12]
5
[0]
0
[4, 3, 2, 0]
2
[0]
0
[1]
1
[43, 32, 27, 21, 20, 19, 19, 15, 13, 9, 8, 7, 5, 4, 1]
9
[35, 29, 24, 22, 16, 8]
6
[8]
1
[8, 5, 2]
2
[4]
1
[14, 8, 6, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0]
4
[18, 15, 11, 11, 9, 8, 3, 0, 0, 0, 0]
6
[13, 5, 4, 1]
3
[7]
1
[59, 36, 35, 29, 22]
5
[20]
1
[15, 4, 4, 2]
3
[18, 10]
2
[33, 32, 26, 22, 17, 13, 13, 12, 11, 8, 4, 0]
9
[53, 51, 6, 5, 4]
4
[0, 0, 0]
0
[17, 17, 15, 12, 11, 11, 9, 9, 8, 8, 6, 4, 2]
8
[7, 4]
2
[1, 1, 0]
1
[15, 14, 10, 8, 5, 4]
5
[30, 28, 26, 24, 23, 22, 19, 11]
8
[13, 10, 8]
3
[9]
1
[56, 51, 49, 47, 44, 38, 36, 36, 34, 34, 33, 31, 31, 30, 30, 29, 29, 28, 27, 27, 27, 26, 19, 18, 15, 15, 14, 13, 13, 13, 11, 8, 7, 4, 3, 2, 2, 2, 2, 1, 1, 0]
22
[15]
1
[39, 32, 31, 26, 26, 22, 10, 10]
8
[16]
1
[1]
1
[3]
1
[46, 15, 14, 10, 5, 4, 4, 3, 2, 2, 1, 0]
5
[2]
1
[0]
0
[18, 6]
2
[12, 12, 6, 4, 3, 3, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0]
4
[12, 8, 5, 3, 0]
3
[40, 1]
1
[2]
1
[5, 0, 0, 0]
1
[73, 57, 38, 32, 28, 28, 27, 26, 25,

2
[1]
1
[2]
1
[0]
0
[10]
1
[3]
1
[48, 32]
2
[23, 11, 8, 4, 3, 1, 0]
4
[2, 0]
1
[13]
1
[2, 1, 1, 0]
1
[38, 28]
2
[3]
1
[12, 10]
2
[7, 4, 2]
2
[6, 2]
2
[43, 42, 36, 19, 17, 14, 12, 11, 9, 9, 8, 3, 2, 1, 1, 0, 0, 0]
9
[0]
0
[15]
1
[18, 10]
2
[12, 4, 1]
2
[24, 18, 10, 8]
4
[41, 32, 18, 16, 9, 7]
6
[7, 6, 6, 5, 3]
4
[11, 4, 1]
2
[7, 4]
2
[154, 79, 77, 67, 46, 39, 34, 28, 28, 27, 26, 25, 22, 22, 19, 18, 17, 17, 16, 14, 13, 12, 12, 10, 9, 9, 9, 3, 1, 1, 1, 1, 0, 0, 0]
17
[11]
1
[4]
1
[17]
1
[60, 20]
2
[96, 36, 34, 29, 26, 20, 12, 8, 7, 7]
8
[1]
1
[1]
1
[5]
1
[9, 8, 3, 3, 1]
3
[43, 38, 37, 36, 36, 31, 30, 29, 23, 17]
10
[24, 21, 11, 9, 9, 0]
5
[2]
1
[0]
0
[21, 13, 11]
3
[2]
1
[43, 35, 17, 11, 11]
5
[41]
1
[120, 32, 27, 19]
4
[1]
1
[15, 11, 8, 6, 5, 4, 2, 1]
5
[17, 16, 13, 13, 10, 9, 8, 7, 7, 6, 3]
7
[1]
1
[0, 0]
0
[2]
1
[1]
1
[2]
1
[4]
1
[45]
1
[1]
1
[1]
1
[3, 2]
2
[5]
1
[0]
0
[10]
1
[18, 14, 12, 8]
4
[16]
1
[17, 5]
2
[0]
0
[3, 3, 2, 0, 0]
2
[8]
1
[1, 1]
1
[0]
0
[26]
1
[4, 0]
1
[6]
1
[3]
1
[99

[11, 9, 6, 5, 2]
4
[20, 13, 9, 7, 4, 2, 0]
4
[21, 17, 14, 12, 7, 3]
5
[20]
1
[0]
0
[47, 38, 0]
2
[22, 21, 18, 14, 13, 12, 12, 10, 8, 8, 5]
8
[0]
0
[10]
1
[19, 17]
2
[44]
1
[0]
0
[33, 14, 5, 3, 1]
3
[6]
1
[1]
1
[22]
1
[22, 14, 12]
3
[11]
1
[13, 13, 12, 10]
4
[5]
1
[6, 6, 5, 4, 3, 2]
4
[18, 17, 15, 14, 4, 0, 0, 0]
4
[26, 10]
2
[2, 1]
1
[37, 20, 10]
3
[8]
1
[2]
1
[17, 16, 16, 15, 15, 15, 14, 14, 13, 12, 10, 10, 9, 8, 8, 8, 7, 6, 4, 3, 2, 2, 2, 1]
10
[19, 18, 7, 2]
3
[7, 4]
2
[3]
1
[28]
1
[26, 16, 14, 13, 13, 12, 7, 6, 4, 4, 3, 3, 2, 1, 1, 1, 1, 0]
7
[32]
1
[4]
1
[33]
1
[30, 29, 11, 2]
3
[8, 3, 3]
3
[2, 2]
2
[3, 3]
2
[46, 42, 33, 30, 28, 27, 27, 27, 24, 22, 19, 18, 15, 13, 11, 11, 8, 2, 2, 2, 1, 0, 0]
13
[25, 10, 3, 0, 0]
3
[73, 33, 32, 29, 21, 18, 14, 14, 13, 11, 11, 9, 9, 8, 8, 8, 7, 7, 6, 6, 6, 5, 3, 2, 1, 1, 1, 1, 0, 0, 0]
11
[6]
1
[10]
1
[12, 12, 6]
3
[30]
1
[30]
1
[5, 2, 2, 0]
2
[6, 3]
2
[1, 0]
1
[9, 6]
2
[1]
1
[36, 16, 15]
3
[23, 21]
2
[3, 0]
1
[12, 5, 2, 1, 1, 0, 0]
2
[3, 3]
2
[3]


1
[58, 46, 39, 39, 36, 32, 31, 31, 30, 27, 21, 20, 20, 19, 19, 15, 14, 9, 6, 1, 0]
15
[0]
0
[4]
1
[7, 4]
2
[14, 8]
2
[18, 0, 0]
1
[0]
0
[18, 10, 7]
3
[0]
0
[7, 3, 1, 1, 0, 0]
2
[2]
1
[7]
1
[50, 35, 33, 26, 18, 7, 5, 5]
6
[2]
1
[2, 2]
2
[9, 7]
2
[1]
1
[27, 20, 18, 17, 17, 16, 16, 15, 14, 9, 6]
9
[5]
1
[5, 3, 1, 1, 0]
2
[9, 5, 2, 2, 0]
2
[3]
1
[6]
1
[17, 3]
2
[3, 2]
2
[0]
0
[13, 5, 5, 4, 4, 2, 2]
4
[6, 3, 1]
2
[35, 30, 24, 20, 7, 7, 5]
6
[0]
0
[22, 21, 16, 8, 4, 0]
4
[44, 44, 15, 6]
4
[27, 7, 6, 3, 1]
3
[1, 0]
1
[0]
0
[20, 7, 2]
2
[186, 40, 38, 23, 14, 8, 5, 3, 3, 0]
6
[17, 16, 15, 7]
4
[19, 10]
2
[21, 10, 5]
3
[37, 32, 32, 9, 6, 2, 1]
5
[8, 7, 6, 5, 4]
4
[20]
1
[14, 9, 6, 5, 5, 5, 5, 4, 3, 2, 1, 1, 0]
5
[1]
1
[41, 41, 36, 32, 29, 28, 24, 24, 22, 22, 19, 15, 15, 14, 13, 11, 11, 10, 9, 6, 3, 2, 1, 0, 0, 0, 0]
14
[8, 5, 5, 2, 2, 1, 1]
3
[14]
1
[4, 4, 2]
2
[25, 3, 1, 1]
2
[11]
1
[3]
1
[10]
1
[22]
1
[31, 27, 25, 21, 18, 17, 14, 7, 4, 3, 1, 0]
7
[4]
1
[4, 4]
2
[0]
0
[44, 30, 29, 20, 17, 14, 9

...

9
[2]
1
[3]
1
[0]
0
[6, 5, 5, 3]
3
[68, 53, 40, 35, 29, 26, 25, 24, 21, 19, 16, 15, 8, 6]
12
[2]
1
[13]
1
[9, 6, 6, 4, 3]
4
[9]
1
[0]
0
[0]
0
[10, 9, 6, 5, 3, 1]
4
[14]
1
[23, 22, 17, 14, 14, 13, 13, 12, 12, 11, 11, 10, 10, 8, 7, 7, 7, 6, 5, 3, 2, 0]
11
[0]
0
[5]
1
[20]
1
[0]
0
[17, 1]
1
[3]
1
[1, 0, 0]
1
[2]
1
[8, 2]
2
[1]
1
[6]
1
[6, 4, 3, 3, 2, 2, 2, 1, 0]
3
[36, 16]
2
[4]
1
[2]
1
[11, 4]
2
[3, 2, 1, 0]
2
[2, 0, 0, 0]
1
[20, 10, 10, 10, 9, 9, 8, 7, 5, 4, 4]
7
[3]
1
[7, 4]
2
[5, 0]
1
[4]
1
[26, 9, 7]
3
[23, 5, 1]
2
[33, 31, 25, 21, 15, 13, 13, 11, 9, 8, 7, 1]
9
[4]
1
[4, 2, 2]
2
[3, 0, 0]
1
[5, 3]
2
[34, 34, 31, 26, 24, 18, 17, 13, 13, 12, 11]
11
[7]
1
[1, 1, 0]
1
[34, 32, 29, 28, 28, 28, 27, 26, 26, 24, 23, 23, 23, 18, 17, 15, 15, 14, 8, 7]
15
[8]
1
[21]
1
[12]
1
[22, 20, 16, 11, 2, 0, 0]
4
[8]
1
[1]
1
[6, 5, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
3
[8]
1
[3, 2]
2
[251, 66, 57, 54, 50, 49, 48, 41, 40, 38, 38, 36, 35, 34, 30, 30, 29, 29, 27, 26, 26, 26, 24, 24, 23, 23, 21, 21, 21, 20

1
[10]
1
[4]
1
[25, 11, 3, 1, 0, 0, 0]
3
[3]
1
[18]
1
[4]
1
[4]
1
[22, 20, 16, 16, 13, 10, 10, 8, 6, 6, 5, 4, 4, 3, 3, 1, 1]
8
[35]
1
[2]
1
[12]
1
[1]
1
[2, 1]
1
[9, 7, 6, 3, 2, 2, 2, 1]
3
[9, 8, 3]
3
[15, 8, 4]
3
[8, 3]
2
[4, 4, 2, 1, 0]
2
[48, 35, 16, 12, 11, 5]
5
[34]
1
[8]
1
[9, 9, 5, 3]
3
[1]
1
[24, 19]
2
[16, 15, 15, 13, 9, 9, 7]
7
[4, 3, 1]
2
[12, 12, 3, 3, 2, 2, 1, 0]
3
[5, 3, 3, 3]
3
[129, 6, 4]
3
[32, 26]
2
[2]
1
[1, 0]
1
[0]
0
[46, 12]
2
[19]
1
[47, 31, 30, 19, 11]
5
[5, 4, 3, 1]
3
[2]
1
[26, 20, 13, 12, 9, 7, 5, 5, 5]
6
[0]
0
[2, 1, 0, 0]
1
[6]
1
[149, 58, 51, 49, 47, 46, 44, 38, 36, 36, 33, 32, 32, 25, 24, 23, 22, 22, 18, 18, 16, 16, 15, 15, 11, 9, 7, 7, 6, 5, 3, 2, 0, 0]
18
[14]
1
[1]
1
[0]
0
[47, 37, 33, 31, 28, 28, 26, 25, 23, 19, 19, 16, 16, 15, 4, 3]
14
[2]
1
[13, 13, 8, 6, 6, 5, 0]
5
[6]
1
[16, 14, 13, 12, 11, 11, 11, 10, 10, 9, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6, 5, 5, 3, 3, 3]
9
[14, 14, 14]
3
[2]
1
[1]
1
[17, 15]
2
[5]
1
[5, 4, 3, 1]
3
[2]
1
[17, 11]
2
[1]
1
[1]
1
[14, 

[10, 9, 7, 6, 5, 4, 2, 0]
5
[4, 4]
2
[0]
0
[12]
1
[2]
1
[2]
1
[10, 2]
2
[2, 1]
1
[20]
1
[24, 5, 4]
3
[45, 26, 23, 22, 20, 17, 16, 16, 15, 15, 14, 13, 13, 10, 9, 7, 5, 2, 0, 0]
13
[1]
1
[4]
1
[18, 14, 5]
3
[0, 0]
0
[25, 16]
2
[20, 11, 6, 6, 5, 4, 2, 2, 0]
5
[40]
1
[6, 3, 2, 1]
2
[40, 29, 26, 26, 26, 24, 24, 20, 17, 16, 7, 5, 5]
10
[28, 4, 4]
3
[36, 32, 28, 25, 25, 24, 20, 18, 17, 17, 16, 15, 14, 14, 10, 9, 8, 8, 6, 6, 6, 6, 6, 6, 5, 5, 5, 4, 4, 4, 3, 2, 1, 1]
14
[7, 4, 2]
2
[2, 2, 1, 0]
2
[1, 0]
1
[14, 6, 3, 3, 1, 1, 1, 0, 0]
3
[41, 40, 37, 32, 30, 28, 27, 27, 25, 24, 24, 23, 21, 20, 18, 16, 12, 9]
16
[28, 9, 3, 1, 1, 1, 1, 0]
3
[0]
0
[3, 2, 0]
2
[3, 3]
2
[14]
1
[40, 26, 23, 22, 21, 20, 18, 16, 13, 11]
10
[3, 2, 0]
2
[8, 8, 4, 3, 3, 3, 2, 2, 1, 1, 1]
3
[0]
0
[62, 55, 29, 29, 26, 23, 23, 21, 19, 19, 17, 16, 16, 13, 10, 6, 6, 5, 4, 4]
13
[5, 1, 1, 1, 0]
1
[4]
1
[6, 5]
2
[14, 10, 3, 3, 1, 0]
3
[44, 37, 14]
3
[30, 28]
2
[2, 1]
1
[8]
1
[4]
1
[42, 28, 13, 12, 11, 3, 2]
5
[2]
1
[1]
1
[7]
1
[1]

[4]
1
[1]
1
[31, 23, 14, 9, 5, 3, 2, 2, 1, 1, 1, 1, 0, 0]
5
[19, 11, 5]
3
[19, 10, 3, 2, 1, 1, 1]
3
[37, 20, 20, 18, 15, 14, 13, 12, 12, 12, 11, 10, 8, 8, 8, 7, 5, 5, 5, 5, 4, 2, 2, 2, 0]
11
[28]
1
[49, 45, 40, 28, 25, 18, 17, 14, 3, 1, 0]
8
[9]
1
[4]
1
[13, 10, 7, 6, 6, 6, 5, 5, 4, 4, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
6
[1]
1
[18, 16, 8, 6, 5, 4, 3, 1]
5
[8]
1
[4]
1
[20, 15, 10, 6]
4
[8, 4]
2
[2]
1
[2]
1
[2]
1
[25, 10]
2
[28, 25, 24, 21, 21, 21, 20, 20, 20, 19, 19, 19, 18, 18, 17, 16, 16, 15, 15, 14, 13, 13, 13, 12, 12, 11, 11, 10, 10, 10, 10, 9, 6, 6, 6, 5, 5, 4, 4, 3, 0]
16
[42, 37, 30, 26, 24, 23, 19, 13, 10, 6, 6, 4]
9
[7]
1
[1]
1
[0]
0
[28, 19, 7]
3
[15, 5, 3, 1]
3
[0]
0
[14]
1
[15, 11, 9, 7, 5, 4, 4, 4, 3]
5
[4]
1
[39, 32, 26, 13, 9, 6, 3, 2]
6
[75, 75, 70, 57, 52, 42, 39, 37, 37, 35, 35, 35, 34, 33, 31, 31, 31, 29, 29, 23]
20
[11]
1
[3, 0]
1
[18, 7]
2
[10]
1
[30, 15, 10, 8, 3, 3, 1, 0]
4
[78, 17]
2
[2]
1
[10, 5, 2, 1]
2
[22, 16, 15, 14, 8]
5
[4]
1
[9, 8, 6, 6, 4, 

[1, 1, 0]
1
[33, 17, 14, 3, 2, 1]
3
[42, 26, 12, 7, 4]
4
[18, 10, 9, 5, 4, 3, 3, 1, 1, 1, 1, 1, 1, 0]
4
[5]
1
[5]
1
[5]
1
[13]
1
[3]
1
[28, 22, 13]
3
[80, 49, 41, 38, 35, 35, 30, 21, 17, 16, 13, 12, 11, 8, 4, 4, 4, 1]
12
[26, 21, 17, 14, 13, 8, 8, 7, 4, 4, 3, 0]
7
[7, 4]
2
[69]
1
[28, 5]
2
[10, 8, 3, 2, 2]
3
[3]
1
[11, 8, 7, 6]
4
[3]
1
[26, 16, 14, 13, 13, 12, 11, 9, 7, 6, 4, 2, 0]
8
[9]
1
[1, 0, 0]
1
[5]
1
[5]
1
[0]
0
[27, 23, 22, 21, 14, 14, 13, 13, 9, 7, 6]
9
[35, 12, 5, 1]
3
[13, 5, 5, 1]
3
[44]
1
[38, 15, 12, 10, 9, 7, 5, 2, 0, 0]
6
[137]
1
[13]
1
[0]
0
[16, 12, 0]
2
[10]
1
[15, 3, 2]
2
[2]
1
[2]
1
[48, 44, 34, 33, 6]
5
[34, 34, 16]
3
[11]
1
[44, 33, 26, 25, 22, 4]
5
[0]
0
[11, 11, 11, 7, 6, 4, 2, 2, 1, 0]
5
[8]
1
[12, 9, 9, 8, 6, 6, 6, 6, 6, 5, 5, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 1, 0]
6
[7]
1
[37]
1
[15]
1
[5, 4]
2
[26, 17, 14, 13, 11, 10, 9, 8]
8
[2, 1]
1
[4]
1
[2, 1]
1
[11, 11]
2
[16, 14, 13, 12, 12, 8, 7, 6, 5, 4, 4, 0, 0, 0]
7
[2]
1
[1]
1
[38, 26, 18, 13, 13, 11, 11, 9

In [ ]:
for i in G:
    #print(i)
    auteurs = []
    for k,v in authors.items():
        #print(v)
        #print(long(i) in v)
        if long(i) in v:
            auteurs.append(k)
    #print(auteurs)
    hlist = [authors_h[j] for j in auteurs]
    if len(hlist) == 0:
        G.node[i]['h-index'] = 1
    #print(hlist)
    else:
        G.node[i]['h-index'] = sum(hlist)/len(hlist)

In [ ]:
print(9305019 in [209085L, 301071L, 9201033L, 9202006L, 9203009L, 9302014L, 9305019L, 9404048L, 9605013L, 9705014L, 9805212L, 9811146L, 9811200L, 9903209L])

In [ ]:
h-index-features = []
for citation in training_set:
        h-index-features.append(G.node[citation[0]]['h-index'] + G.node[citation[1]]['h-index'])

In [ ]:
print(h-index-features)